In [1]:
# Imports the 'os' module to work with the file system.
import os

# Imports necessary functions and libraries for image processing using Keras and TensorFlow.
from keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

# Provides a progress bar for tracking data processing progress.
from tqdm.notebook import tqdm  


# Imports the ResNet50 model and preprocessing functions from TensorFlow's Keras module.
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input 

# Imports the Model class from TensorFlow's Keras models module.
from tensorflow.keras.models import Model

# Imports the 'pickle' module for serializing Python objects.
import pickle 


In [2]:
WORKING_DIR = '/Users/aman/Downloads/Flicker'

In [3]:
# Loads the ResNet50 model pre-trained on ImageNet data.
model = ResNet50(weights='imagenet', input_shape=(224, 224, 3))

# Restructures the model by removing the last two layers.
# The last two layers are typically the classification layers, leaving the feature representation intact.
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

# Prints a summary of the modified model, providing information about its architecture.
print(model.summary())



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

                                                                                                  
 conv2_block3_1_conv (Conv2  (None, 56, 56, 64)           16448     ['conv2_block2_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv2_block3_1_bn (BatchNo  (None, 56, 56, 64)           256       ['conv2_block3_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv2_block3_1_relu (Activ  (None, 56, 56, 64)           0         ['conv2_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv2_blo

 ation)                                                                                           
                                                                                                  
 conv3_block2_3_conv (Conv2  (None, 28, 28, 512)          66048     ['conv3_block2_2_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv3_block2_3_bn (BatchNo  (None, 28, 28, 512)          2048      ['conv3_block2_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv3_block2_add (Add)      (None, 28, 28, 512)          0         ['conv3_block1_out[0][0]',    
                                                                     'conv3_block2_3_bn[0][0]']   
          

 conv4_block1_2_conv (Conv2  (None, 14, 14, 256)          590080    ['conv4_block1_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv4_block1_2_bn (BatchNo  (None, 14, 14, 256)          1024      ['conv4_block1_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block1_2_relu (Activ  (None, 14, 14, 256)          0         ['conv4_block1_2_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv4_block1_0_conv (Conv2  (None, 14, 14, 1024)         525312    ['conv3_block4_out[0][0]']    
 D)       

                                                                                                  
 conv4_block3_out (Activati  (None, 14, 14, 1024)         0         ['conv4_block3_add[0][0]']    
 on)                                                                                              
                                                                                                  
 conv4_block4_1_conv (Conv2  (None, 14, 14, 256)          262400    ['conv4_block3_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv4_block4_1_bn (BatchNo  (None, 14, 14, 256)          1024      ['conv4_block4_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_blo

 D)                                                                                               
                                                                                                  
 conv4_block6_3_bn (BatchNo  (None, 14, 14, 1024)         4096      ['conv4_block6_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block6_add (Add)      (None, 14, 14, 1024)         0         ['conv4_block5_out[0][0]',    
                                                                     'conv4_block6_3_bn[0][0]']   
                                                                                                  
 conv4_block6_out (Activati  (None, 14, 14, 1024)         0         ['conv4_block6_add[0][0]']    
 on)                                                                                              
          

 conv5_block3_1_relu (Activ  (None, 7, 7, 512)            0         ['conv5_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv5_block3_2_conv (Conv2  (None, 7, 7, 512)            2359808   ['conv5_block3_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv5_block3_2_bn (BatchNo  (None, 7, 7, 512)            2048      ['conv5_block3_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv5_block3_2_relu (Activ  (None, 7, 7, 512)            0         ['conv5_block3_2_bn[0][0]']   
 ation)   

Extration Image Features

In [4]:

# Creates an empty dictionary to store image features and a list for skipped image names.
features = {}
skipped_images = []

# Specifies the directory where the image files are located.
directory = os.path.join(WORKING_DIR, 'images')

# Iterates through the list of files in the specified directory.
for img_name in tqdm(os.listdir(directory)):
    # Checks if the file is an image file based on its file extension.
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        try:
            # Loads the image from the file and prints a processing message.
            img_path = os.path.join(directory, img_name)
            print("Processing image:", img_name)
            image = load_img(img_path, target_size=(224, 224))
            
            # Converts the image pixels to a numpy array.
            image = img_to_array(image)
            
            # Reshapes the image data to match the model's input shape.
            image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            
            # Preprocesses the image for use with the model.
            image = preprocess_input(image)
            
            # Extracts features from the image using the model.
            feature = model.predict(image, verbose=0)
            
            # Retrieves the image ID (filename without extension) and stores the feature.
            image_id = img_name.split('.')[0]
            features[image_id] = feature
            print("Image processed successfully:", img_name)
            
        except Exception as e:
            
            # Handles errors during image processing and logs them.
            print("Error processing image:", img_name)
            print("Error message:", str(e))
            
            # Stores the name of the skipped image in the 'skipped_images' list.
            skipped_images.append(img_name)

# Prints the list of skipped images, if any.
print("Skipped images:", skipped_images)


  0%|          | 0/8091 [00:00<?, ?it/s]

Processing image: 2387197355_237f6f41ee.jpg
Image processed successfully: 2387197355_237f6f41ee.jpg
Processing image: 2609847254_0ec40c1cce.jpg
Image processed successfully: 2609847254_0ec40c1cce.jpg
Processing image: 2046222127_a6f300e202.jpg
Image processed successfully: 2046222127_a6f300e202.jpg
Processing image: 2853743795_e90ebc669d.jpg
Image processed successfully: 2853743795_e90ebc669d.jpg
Processing image: 2696951725_e0ae54f6da.jpg
Image processed successfully: 2696951725_e0ae54f6da.jpg
Processing image: 3421131122_2e4bde661e.jpg
Image processed successfully: 3421131122_2e4bde661e.jpg
Processing image: 3229730008_63f8ca2de2.jpg
Image processed successfully: 3229730008_63f8ca2de2.jpg
Processing image: 3220009216_10f088185e.jpg
Image processed successfully: 3220009216_10f088185e.jpg
Processing image: 3415578043_03d33e6efd.jpg
Image processed successfully: 3415578043_03d33e6efd.jpg
Processing image: 3437273677_47d4462974.jpg
Image processed successfully: 3437273677_47d4462974.jpg


Image processed successfully: 3335547029_74d620fa6c.jpg
Processing image: 278496691_c1fd93e2d8.jpg
Image processed successfully: 278496691_c1fd93e2d8.jpg
Processing image: 268654674_d29e00b3d0.jpg
Image processed successfully: 268654674_d29e00b3d0.jpg
Processing image: 2896640216_761a47f006.jpg
Image processed successfully: 2896640216_761a47f006.jpg
Processing image: 2339946012_06bd480ab8.jpg
Image processed successfully: 2339946012_06bd480ab8.jpg
Processing image: 2319175397_3e586cfaf8.jpg
Image processed successfully: 2319175397_3e586cfaf8.jpg
Processing image: 3591457224_88281dd04f.jpg
Image processed successfully: 3591457224_88281dd04f.jpg
Processing image: 1317292658_ba29330a0b.jpg
Image processed successfully: 1317292658_ba29330a0b.jpg
Processing image: 2789350645_96a2535b4d.jpg
Image processed successfully: 2789350645_96a2535b4d.jpg
Processing image: 2612608861_92beaa3d0b.jpg
Image processed successfully: 2612608861_92beaa3d0b.jpg
Processing image: 3653764864_225958c9c1.jpg
Imag

Image processed successfully: 468141298_3154d717e1.jpg
Processing image: 2458006588_754c4aa09c.jpg
Image processed successfully: 2458006588_754c4aa09c.jpg
Processing image: 3147758035_e8a70818cb.jpg
Image processed successfully: 3147758035_e8a70818cb.jpg
Processing image: 2825540754_5e0c13e6b8.jpg
Image processed successfully: 2825540754_5e0c13e6b8.jpg
Processing image: 410413536_11f1127c46.jpg
Image processed successfully: 410413536_11f1127c46.jpg
Processing image: 3423802527_94bd2b23b0.jpg
Image processed successfully: 3423802527_94bd2b23b0.jpg
Processing image: 241346260_f50d57b517.jpg
Image processed successfully: 241346260_f50d57b517.jpg
Processing image: 516648762_0cff84ea97.jpg
Image processed successfully: 516648762_0cff84ea97.jpg
Processing image: 3112821789_1f7c3bbb99.jpg
Image processed successfully: 3112821789_1f7c3bbb99.jpg
Processing image: 276356412_dfa01c3c9e.jpg
Image processed successfully: 276356412_dfa01c3c9e.jpg
Processing image: 2964438493_413667c04a.jpg
Image pro

Image processed successfully: 2282260240_55387258de.jpg
Processing image: 3613264553_97b687f172.jpg
Image processed successfully: 3613264553_97b687f172.jpg
Processing image: 3654869593_c8599a8e20.jpg
Image processed successfully: 3654869593_c8599a8e20.jpg
Processing image: 2059616165_b7c99c1009.jpg
Image processed successfully: 2059616165_b7c99c1009.jpg
Processing image: 2367139509_1ee4530b28.jpg
Image processed successfully: 2367139509_1ee4530b28.jpg
Processing image: 3046430047_d7b10123d0.jpg
Image processed successfully: 3046430047_d7b10123d0.jpg
Processing image: 2831578193_3c997ae330.jpg
Image processed successfully: 2831578193_3c997ae330.jpg
Processing image: 3420338549_bd78d35243.jpg
Image processed successfully: 3420338549_bd78d35243.jpg
Processing image: 3490517179_76dbd690de.jpg
Image processed successfully: 3490517179_76dbd690de.jpg
Processing image: 1095590286_c654f7e5a9.jpg
Image processed successfully: 1095590286_c654f7e5a9.jpg
Processing image: 2140182410_8e2a06fbda.jpg


...

Image processed successfully: 497579819_f91b26f7d3.jpg
Processing image: 211981411_e88b8043c2.jpg
Image processed successfully: 211981411_e88b8043c2.jpg
Processing image: 3026102616_3cf350af9e.jpg
Image processed successfully: 3026102616_3cf350af9e.jpg
Processing image: 765091078_a8a11c6f9e.jpg
Image processed successfully: 765091078_a8a11c6f9e.jpg
Processing image: 3425853460_bfcd0b41f6.jpg
Image processed successfully: 3425853460_bfcd0b41f6.jpg
Processing image: 3185787277_b412d7f5b7.jpg
Image processed successfully: 3185787277_b412d7f5b7.jpg
Processing image: 425088533_a460dc4617.jpg
Image processed successfully: 425088533_a460dc4617.jpg
Processing image: 543326592_70bd4d8602.jpg
Image processed successfully: 543326592_70bd4d8602.jpg
Processing image: 3296500180_0d7a6650dc.jpg
Image processed successfully: 3296500180_0d7a6650dc.jpg
Processing image: 2878272032_fda05ffac7.jpg
Image processed successfully: 2878272032_fda05ffac7.jpg
Processing image: 2239938351_43c73c887c.jpg
Image pro

Image processed successfully: 2429272699_8a9699775e.jpg
Processing image: 3226796100_08c49cfa5c.jpg
Image processed successfully: 3226796100_08c49cfa5c.jpg
Processing image: 2101128963_fdf8b2a0d7.jpg
Image processed successfully: 2101128963_fdf8b2a0d7.jpg
Processing image: 3041348852_872c027c16.jpg
Image processed successfully: 3041348852_872c027c16.jpg
Processing image: 1813777902_07d1d4b00c.jpg
Image processed successfully: 1813777902_07d1d4b00c.jpg
Processing image: 2269795781_46a296aa21.jpg
Image processed successfully: 2269795781_46a296aa21.jpg
Processing image: 249394748_2e4acfbbb5.jpg
Image processed successfully: 249394748_2e4acfbbb5.jpg
Processing image: 161905204_247c6ca6de.jpg
Image processed successfully: 161905204_247c6ca6de.jpg
Processing image: 2350400382_ced2b6c91e.jpg
Image processed successfully: 2350400382_ced2b6c91e.jpg
Processing image: 1392272228_cf104086e6.jpg
Image processed successfully: 1392272228_cf104086e6.jpg
Processing image: 2752043092_f48ebfeaa2.jpg
Imag

Image processed successfully: 3331009729_d3b14738e6.jpg
Processing image: 453473508_682c0a7189.jpg
Image processed successfully: 453473508_682c0a7189.jpg
Processing image: 3289817083_4e78e1c05a.jpg
Image processed successfully: 3289817083_4e78e1c05a.jpg
Processing image: 3320680380_b0d38b3b4a.jpg
Image processed successfully: 3320680380_b0d38b3b4a.jpg
Processing image: 3036382555_30b7312cf3.jpg
Image processed successfully: 3036382555_30b7312cf3.jpg
Processing image: 3111482098_11c0f4f309.jpg
Image processed successfully: 3111482098_11c0f4f309.jpg
Processing image: 3341782693_426bf7139b.jpg
Image processed successfully: 3341782693_426bf7139b.jpg
Processing image: 2537119659_fa01dd5de5.jpg
Image processed successfully: 2537119659_fa01dd5de5.jpg
Processing image: 3214100656_80cda1b86b.jpg
Image processed successfully: 3214100656_80cda1b86b.jpg
Processing image: 3326273086_e09e845185.jpg
Image processed successfully: 3326273086_e09e845185.jpg
Processing image: 2517082705_93bc9f73ec.jpg
Im

Image processed successfully: 3585488964_1467945775.jpg
Processing image: 1342766791_1e72f92455.jpg
Image processed successfully: 1342766791_1e72f92455.jpg
Processing image: 2277081067_d2b4c98bce.jpg
Image processed successfully: 2277081067_d2b4c98bce.jpg
Processing image: 2564888404_b57f89d3c7.jpg
Image processed successfully: 2564888404_b57f89d3c7.jpg
Processing image: 445148321_9f2f3ac711.jpg
Image processed successfully: 445148321_9f2f3ac711.jpg
Processing image: 1287920676_d21a0b289b.jpg
Image processed successfully: 1287920676_d21a0b289b.jpg
Processing image: 576093768_e78f91c176.jpg
Image processed successfully: 576093768_e78f91c176.jpg
Processing image: 1991806812_065f747689.jpg
Image processed successfully: 1991806812_065f747689.jpg
Processing image: 3360930596_1e75164ce6.jpg
Image processed successfully: 3360930596_1e75164ce6.jpg
Processing image: 514990193_2d2422af2c.jpg
Image processed successfully: 514990193_2d2422af2c.jpg
Processing image: 3150380412_7021e5444a.jpg
Image 

Image processed successfully: 150387174_24825cf871.jpg
Processing image: 3436418401_b00ceb27c0.jpg
Image processed successfully: 3436418401_b00ceb27c0.jpg
Processing image: 2466420387_86fe77c966.jpg
Image processed successfully: 2466420387_86fe77c966.jpg
Processing image: 3622216490_1314a58b66.jpg
Image processed successfully: 3622216490_1314a58b66.jpg
Processing image: 514222303_cb98584536.jpg
Image processed successfully: 514222303_cb98584536.jpg
Processing image: 3694093650_547259731e.jpg
Image processed successfully: 3694093650_547259731e.jpg
Processing image: 3165936115_cb4017d94e.jpg
Image processed successfully: 3165936115_cb4017d94e.jpg
Processing image: 2507312812_768b53b023.jpg
Image processed successfully: 2507312812_768b53b023.jpg
Processing image: 347186933_880caaf53b.jpg
Image processed successfully: 347186933_880caaf53b.jpg
Processing image: 3655773435_c234e94820.jpg
Image processed successfully: 3655773435_c234e94820.jpg
Processing image: 2864340145_d28b842faf.jpg
Image

Serializes and stores the 'features' dictionary into a file named 'inception_features1K.pkl'

In [5]:
# located in the working directory using binary write mode ('wb').
pickle.dump(features, open(os.path.join(WORKING_DIR, 'resnet_features.pkl'), 'wb'))

Loads serialized features from the 'inception_features1K.pkl' file located in the working directory

In [6]:
# using binary read mode ('rb') and assigns them to the 'features' variable.
with open(os.path.join(WORKING_DIR, 'resnet_features.pkl'), 'rb') as f:
    features = pickle.load(f)

In [7]:
# Calculates the length (number of items) in the 'features' dictionary and returns the result.
feature_count = len(features)
print(feature_count)

8091


In [8]:
# Print the first few feature values
for image_id, feature in list(features.items())[:5]:
    print(f"Image ID: {image_id}, Feature shape: {feature.shape}")

Image ID: 2387197355_237f6f41ee, Feature shape: (1, 2048)
Image ID: 2609847254_0ec40c1cce, Feature shape: (1, 2048)
Image ID: 2046222127_a6f300e202, Feature shape: (1, 2048)
Image ID: 2853743795_e90ebc669d, Feature shape: (1, 2048)
Image ID: 2696951725_e0ae54f6da, Feature shape: (1, 2048)


In [10]:
features['2387197355_237f6f41ee']

array([[1.6974244 , 3.476214  , 0.        , ..., 0.36919546, 0.2910824 ,
        0.57165766]], dtype=float32)